### Batches, Batch Normalization and Dropout

In this workbook you can experiment what you learnt about how to make batches out of your data, how to perform batch normalization and dropout

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Load the data from data/batches_norm_drop.csv, then take a look at them.

In [4]:
# YOUR CODE HERE
df=pd.read_csv('data/batches_norm_drop.csv', header=None)
df



,0,1,2
0,0.350140,4.248592,0
1,0.950728,3.528855,0
2,1.371517,3.149416,0
3,0.268221,4.337209,0
4,1.881996,1.515387,0
...,...,...,...
745,-3.425455,3.349783,2
746,-1.513002,2.789840,2
747,-1.070356,3.484981,2
748,-2.970848,3.443924,2


Now you are ready to code your own function to create batches. If needed rewatch the video we provided in Eduflow.

**Extra challange:**    Are you able to split between train and test _**without**_ using sklearn?

In [ ]:
def batchify():
#YOUR CODE HERE
    pass

It's time to create your model! Remember to include the new tricks you learnt (batch normalization and dropout)!

In [ ]:
# YOUR CODE HERE

Train your model and evaluate it. **Extra challenge**: try to figure out how you can tell if batch norm and dropout are effective

In [ ]:
# YOUR CODE HERE